# Mount Drive

In [1]:
#Import drive
from google.colab import drive
#Mount Google Drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('drive/My Drive/Work/Thesis_Julien_Dejasmin/Work/code/Binary_activations_V2/MNIST_binary_V2')

In [3]:
!ls

config.py   distributions  models	requirements.txt  utils
data	    experiments    __pycache__	results		  visualize
DataLoader  logs	   README.md	trained_models


# Import:

In [0]:
import torch.optim as optim
from DataLoader.dataLoaders import get_omniglot_dataloaders_v1
from utils.models import NoBinaryNetOmniglotClassification, BinaryNetOmniglotClassification
from visualize.viz import show_som_examples
from utils.training import training, test, gpu_config
from torch import load
import torch
# from torchsummary import summary

# Parameters:

In [0]:
# parameters default values
lr = 1e-3
# momentum = 0.9
nb_epoch = 50
batch_size_train = 64
batch_size_test = 128

# Dataset:

In [6]:
# Omniglto Dataset
train_loader, test_loader = get_omniglot_dataloaders_v1(batch_size_train, batch_size_test)

Files already downloaded and verified
Number of training examples: 272
Number of testing examples: 16


# Model:

In [0]:
# Model, activation type, estimator type
def get_my_model(binary, stochastic, reinforce, first_conv_layer, second_conv_layer, 
                 third_conv_layer, fourth_conv_layer):

    if binary:
        if stochastic:
            mode = 'Stochastic'
            names_model = 'Omniglot_classif/Stochastic'
        else:
            mode = 'Deterministic'
            names_model = 'Omniglot_classif/Deterministic'
        if reinforce:
            estimator = 'REINFORCE'
            names_model += '_REINFORCE'
        else:
            estimator = 'ST'
            names_model += '_ST'
        if first_conv_layer:
            names_model += '_first_conv_binary'
        if second_conv_layer:
            names_model += '_second_conv_binary'
        if third_conv_layer:
            names_model += '_third_conv_binary'
        if fourth_conv_layer:
            names_model += '_fourth_conv_binary'
        model = BinaryNetOmniglotClassification(first_conv_layer=first_conv_layer, second_conv_layer=second_conv_layer,
                                        third_conv_layer=third_conv_layer, fourth_conv_layer=fourth_conv_layer,
                                        mode=mode, estimator=estimator)
    else:
        model = NoBinaryNetOmniglotClassification()
        names_model = 'Omniglot_classif/NonBinaryNet'
        mode = None
        estimator = None
 
    # gpu config:
    model, use_gpu = gpu_config(model)
    return model, names_model, use_gpu

# No binary Model:

## Create model

In [8]:
# Parameters
slope_annealing = False
reinforce = False
stochastic = False
binary = False
plot_result = True
first_conv_layer = False
second_conv_layer = False
third_conv_layer = False
fourth_conv_layer = False
omniglot = True

model, name_model, use_gpu = get_my_model(binary, stochastic, reinforce, first_conv_layer, 
                                 second_conv_layer, third_conv_layer, fourth_conv_layer)

print(name_model)

no gpu available !
Omniglot_classif/NonBinaryNet


In [9]:
print(model)

NoBinaryNetOmniglotClassification(
  (layer1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxPool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (act_layer1): ReLU()
  (layer2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxPool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (act_layer2): ReLU()
  (layer3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxPool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (act_layer3): ReLU()
  (layer4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm4): BatchNorm2d(64

## Training:

In [0]:
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

train_loss, train_acc, val_loss, val_acc = training(use_gpu, model, name_model, nb_epoch, train_loader, test_loader,
                                                    optimizer, plot_result, slope_annealing)

# Epoch : 0 - Slope : 1.0


## Test:

In [0]:
# test
model.load_state_dict(load('trained_models/' + name_model + '.pt', map_location=torch.device('cpu')))
test_loss, test_acc = test(use_gpu, model, test_loader)

/content/drive/My Drive/Work/Thesis_Julien_Dejasmin/Work/code/Binary_activations_V2/MNIST_binary_V2/utils/training.py:131: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Test set: Average loss: 0.4507, Accuracy: 1698/1928 (88%)



# Omniglot classification Network with binary first conv layer:

In [0]:
# Parameters
slope_annealing = False
reinforce = False
stochastic = True
binary = False
plot_result = True
first_conv_layer = True
second_conv_layer = False
third_conv_layer = False
fourth_conv_layer = False
omniglot = True

model, name_model, use_gpu = get_my_model(binary, stochastic, reinforce, first_conv_layer, 
                                 second_conv_layer, third_conv_layer, fourth_conv_layer)

print(name_model)

use 1 gpu who named: Tesla P100-PCIE-16GB
Omniglot_classif/NonBinaryNet


In [0]:
print(model)

NoBinaryNetOmniglotClassification(
  (layer1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxPool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (act_layer1): ReLU()
  (layer2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxPool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (act_layer2): ReLU()
  (layer3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxPool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (act_layer3): ReLU()
  (layer4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm4): BatchNorm2d(64

In [0]:
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

train_loss, train_acc, val_loss, val_acc = training(use_gpu, model, name_model, nb_epoch, train_loader, test_loader,
                                                    optimizer, plot_result, slope_annealing)

# Epoch : 0 - Slope : 1.0


In [0]:
# test
model.load_state_dict(load('trained_models/' + name_model + '.pt', map_location=torch.device('cpu')))
test_loss, test_acc = test(use_gpu, model, test_loader)

In [0]:
# Parameters
slope_annealing = False
reinforce = False
stochastic = True
binary = False
plot_result = True
first_conv_layer = False
second_conv_layer = True
third_conv_layer = False
fourth_conv_layer = False
omniglot = True

model, name_model, use_gpu = get_my_model(binary, stochastic, reinforce, first_conv_layer, 
                                 second_conv_layer, third_conv_layer, fourth_conv_layer)

print(name_model)
print(model)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

train_loss, train_acc, val_loss, val_acc = training(use_gpu, model, name_model, nb_epoch, train_loader, test_loader,
                                                    optimizer, plot_result, slope_annealing)

In [0]:
# Parameters
slope_annealing = False
reinforce = False
stochastic = True
binary = False
plot_result = True
first_conv_layer = False
second_conv_layer = False
third_conv_layer = True
fourth_conv_layer = False
omniglot = True

model, name_model, use_gpu = get_my_model(binary, stochastic, reinforce, first_conv_layer, 
                                 second_conv_layer, third_conv_layer, fourth_conv_layer)

print(name_model)
print(model)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

train_loss, train_acc, val_loss, val_acc = training(use_gpu, model, name_model, nb_epoch, train_loader, test_loader,
                                                    optimizer, plot_result, slope_annealing)

In [0]:
# Parameters
slope_annealing = False
reinforce = False
stochastic = True
binary = False
plot_result = True
first_conv_layer = False
second_conv_layer = False
third_conv_layer = False
fourth_conv_layer = True
omniglot = True

model, name_model, use_gpu = get_my_model(binary, stochastic, reinforce, first_conv_layer, 
                                 second_conv_layer, third_conv_layer, fourth_conv_layer)

print(name_model)
print(model)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

train_loss, train_acc, val_loss, val_acc = training(use_gpu, model, name_model, nb_epoch, train_loader, test_loader,
                                                    optimizer, plot_result, slope_annealing)